In [4]:
import mdtraj as md
import numpy as np
from scipy.stats import ttest_ind_from_stats

from tqdm import tqdm
from pathlib import Path
import os
from natsort import natsorted
import pandas as pd
from addict import Dict as Adict

import pyemma as pm
import deeptime as dt
import deeptime.markov.msm as msm
import deeptime.markov.hmm as hmm

from deeptime.plots import plot_implied_timescales, plot_energy2d, plot_contour2d_from_xyz
from deeptime.markov.sample import *
from deeptime.markov import TransitionCountEstimator
from deeptime.util import energy2d

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns

from funcs_build_msm import _tica, get_data 
from funcs_validate_msm import *
from funcs_sample import *
from funcs_plotting import *

In [22]:
protein = 'abl'
study_name = 'n_clusters'
features = ['dbdist', 'dbdihed', 'aloop', 'achelix']
hp_id = 2
bs_id = 0

In [26]:
data_path = Path(f'data_{protein}/msm/{study_name}')
msm_path = data_path/f'{hp_id}'
summary_f = data_path/f'{study_name}.h5'

hps = pd.read_hdf(summary_f, key='hps')
raw = pd.read_hdf(summary_f, key='result_raw')
hp_dict = Adict(hps[hps.hp_id == hp_id].to_dict('records')[0])

raw.loc[(raw.hp_id == hp_id) & (raw.bs == bs_id)]

,hp_id,bs,is_sparse,t2,t3,t4,t5,t6,t7,t8,...,vamp2eq_12,vamp2eq_13,vamp2eq_14,vamp2eq_15,vamp2eq_16,vamp2eq_17,vamp2eq_18,vamp2eq_19,vamp2eq_20,vamp2eq_21
0,2,0,True,1.833997e+06,1.266339e+06,77306.295168,28905.215896,9145.507739,7924.966831,3852.193645,...,11.542568,12.415974,13.225425,14.034613,14.84017,15.632894,16.422647,17.189757,17.932025,18.659759


In [13]:
savedir = Path(f'data_{protein}/msm/validation/{study_name}_hp{hp_id}_bs{bs_id}')
savedir.mkdir(exist_ok=True, parents=True)